In [1]:
# columns
# ID (to be created in SQL)
# member_id
# rx_code
# pharmacy_id
# network_status
# cost
# payout
# date_of_service

In [2]:
import datetime as dt
import numpy as np
import pandas as pd
import random 

In [3]:
# create dataframe
rx_claims= pd.DataFrame()

In [4]:
# 1. member_id

# import member_ID column from members table from MySQL to get list of IDs
# SQL connection
from sqlalchemy import create_engine
import pymysql

# estabish connection
engine = create_engine('mysql+pymysql://root:funny20!@#@localhost')

In [5]:
engine.execute("USE health_company;")

In [6]:
# read Member_ID column into a dataframe
df_member_id = pd.read_sql_query("select Member_ID from members;", con=engine)

In [7]:
member_id = list(df_member_id["Member_ID"])

In [8]:
# create list of 20000 randomly selected member ids
member_id = [random.choice(member_id) for i in range(20000)]

In [9]:
len(member_id)

20000

In [10]:
# add member_ID to dataframe
rx_claims["Member_ID"] = member_id

In [11]:
# 2. pharmacy_id

# import Pharmacy_ID , Network_Status column from 'pharmacies' table in MySQL
df_pharm_id = pd.read_sql_query("select Pharmacy_ID , Network_Status from pharmacies;", con=engine)

In [12]:
# convert Pharmacy_ID column to list and randomize in 20000 item list
pharm_id = list(df_pharm_id['Pharmacy_ID'] )
pharm_id = [random.choice(pharm_id) for i in range(20000)]

In [13]:
#insert into dataframe
rx_claims['Pharmacy_ID'] = pharm_id

In [14]:
# 3. Network_Status

# merge dataframes to inner join Network_Status to procedure_claims
rx_claims = pd.merge(rx_claims, df_pharm_id, on='Pharmacy_ID')

In [15]:
# 4. rx_code and 5. cost

In [17]:
# Change 'Price' to 'Cost'in drugs table in MySQL
engine.execute('ALTER TABLE drugs RENAME COLUMN Price TO Cost;')

In [18]:
# import Code, Cost from procedures table in sql 
df_codes_cost = pd.read_sql_query('select NDC_Code, Cost from drugs;', con=engine)

In [19]:
codes = list(df_codes_cost['NDC_Code'])

In [20]:
# create list of 20000 randomly selected codes
codes = [random.choice(codes) for i in range(20000)]

In [21]:
# add codes to dataframe
rx_claims["NDC_Code"] = codes

In [22]:
# 5. inner join cost column to dataframe with .merge
rx_claims = pd.merge(rx_claims, df_codes_cost, on="NDC_Code")

In [23]:
# check length of rx_claims
len(rx_claims)

20997

In [24]:
# remove rows to get to 20000
while len(rx_claims) > 20000:
    rx_claims = rx_claims.drop(rx_claims.index[-1])

In [25]:
# check length of rx_claims
len(rx_claims)

20000

In [26]:
# 6. payout

# these will be weighted random fractions of the cost columns. We will assume 80/20 cost sharing, meaning .8 will
# have the highest weight among the fractions. 
def weighted_choice(objects, weights):
    """ returns a random element from the sequence of 'objects', 
        the likelihood of the objects is weighted according 
        to the sequence of 'weights', i.e. percentages."""

    weights = np.array(weights, dtype=np.float64)
    sum_of_weights = weights.sum()
    # standardization:
    np.multiply(weights, 1 / sum_of_weights, weights)
    weights = weights.cumsum()
    x = random.random()
    for i in range(len(weights)):
        if x < weights[i]:
            return objects[i]

# call weighted_choice function, where both 0.8 and a random number between 0,1 are equally likey to be chosen
rates = [weighted_choice([0.8, round(random.uniform(0,1),2)], [0.5, 0.5]) for i in range(20000)]

In [27]:
# initialize empty list payout and iterate through dataframe
payout = []
for i,j in rx_claims.iterrows():
    # if in-network, multiply Cost and rates to get payout
    if j[2] == "In-network":
        payout += [rx_claims['Cost'][i]*rates[i]]
    # otherwise, enter 0
    else:
        payout +=[0]

In [28]:
# insert into dataframe
rx_claims['Payout'] = payout

In [29]:
# 7. date of service

In [30]:
# import enrollment date from MySQL to ensure that DOS comes after enrollment date
enr_list = list(pd.read_sql_query('select Enrollment_Date from members;', con=engine)['Enrollment_Date'])

In [31]:
# define function to ensure March 20 2020 is not exceeded
def march_20(a):
    a += dt.timedelta(days=random.randint(1,100))
    while a > dt.date(2020, 3, 20):
        a -= dt.timedelta(days=1)
    return a


dos = [march_20(i) for i in enr_list*2]

In [32]:
# insert into dataframe
rx_claims["Date_Of_Service"] = dos

In [33]:
rx_claims.head()

,Member_ID,Pharmacy_ID,Network_Status,NDC_Code,Cost,Payout,Date_Of_Service
0,104005,300544,In-network,0268-6196,34,26.18,2017-12-11
1,104005,300544,In-network,0268-6196,50,40.00,2014-10-09
2,100137,300119,In-network,0268-6196,34,27.20,2013-09-08
3,100137,300119,In-network,0268-6196,50,40.00,2017-05-19
4,107643,300544,In-network,24338-120,9,7.20,2015-03-21


In [34]:
# create table in MySQL
engine.execute('CREATE TABLE rx_claims (\
                  Rx_Claim_ID INT NOT NULL AUTO_INCREMENT,\
                  Member_ID INT,\
                  Pharmacy_ID INT,\
                  Network_Status VARCHAR(50),\
                  NDC_Code VARCHAR(12),\
                  Cost INT,\
                  Payout DECIMAL(6,2),\
                  Date_Of_Service DATE,\
                  PRIMARY KEY(Rx_Claim_ID)\
              ) AUTO_INCREMENT = 500000;')

In [35]:
rx_claims.to_sql('rx_claims', con=engine, if_exists='append', index=False)

In [36]:
engine.execute('select * from rx_claims limit 10;').fetchall()

[(500000, 104005, 300544, 'In-network', '0268-6196', 34, Decimal('26.18'), datetime.date(2017, 12, 11)),
 (500001, 104005, 300544, 'In-network', '0268-6196', 50, Decimal('40.00'), datetime.date(2014, 10, 9)),
 (500002, 100137, 300119, 'In-network', '0268-6196', 34, Decimal('27.20'), datetime.date(2013, 9, 8)),
 (500003, 100137, 300119, 'In-network', '0268-6196', 50, Decimal('40.00'), datetime.date(2017, 5, 19)),
 (500004, 107643, 300544, 'In-network', '24338-120', 9, Decimal('7.20'), datetime.date(2015, 3, 21)),
 (500005, 104792, 300864, 'In-network', '24338-120', 9, Decimal('7.20'), datetime.date(2020, 3, 20)),
 (500006, 107044, 300544, 'In-network', '65044-1006', 14, Decimal('11.20'), datetime.date(2011, 6, 17)),
 (500007, 101448, 300544, 'In-network', '0268-1505', 53, Decimal('22.79'), datetime.date(2016, 3, 31)),
 (500008, 104771, 300622, 'In-network', '0268-1505', 53, Decimal('43.46'), datetime.date(2014, 5, 19)),
 (500009, 103858, 300544, 'In-network', '63323-262', 650, Decimal('

In [37]:
engine.dispose()